In [11]:
import os
import pandas as pd
from tqdm import tqdm
from openai import OpenAI
os.environ['OPENAI_API_KEY'] = 'your-api-key'

In [7]:
client = OpenAI()

def get_ans(question, context):
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant for answering questions based on provided context. The context will be provided as 6 snippets of scientific articles. Start the answer with 'Answer: '."},
            {
                "role": "user",
                "content": f"Context: {context} \n Question: {question}"
            }
        ]
    )
    return completion.choices[0].message.content

In [8]:
df = pd.read_csv("../../data/llm_eval/real_outputs.csv")

In [9]:
ground_truths = []

In [15]:
for q, ctx in tqdm(df[["question", "context"]].values[:60], desc="Processing Questions"):
    ground_truths.append(get_ans(q, ctx))

Processing Questions: 100%|██████████| 60/60 [04:16<00:00,  4.27s/it]


In [23]:
ground_truths_df = [gt[8:] for gt in ground_truths]
df["ground_truth"] = ground_truths_df * 4

In [24]:
df

,llm,question,context,answer,ex_time,ground_truth
0,qwen2.5:3b,"What does the term ""learn to optimize"" mean?\n","Document 1: \n NatlSciRev ,2024,Vol.11,nwae132...","The term ""learn to optimize"" (L2O) refers to a...",2.757843,"The term ""learn to optimize"" (L2O) refers to a..."
1,qwen2.5:3b,Please give some examples of metaheuristics.\n,Document 1: \n usually provide only sub-optima...,Metaheuristics are high-level methodologies or...,2.751359,Examples of metaheuristics include Simulated A...
2,qwen2.5:3b,"What is the ""no free lunch"" theorem about?\n",Document 1: \n IEEE TRANSACTIONS ON EVOLUTIONA...,"The ""No Free Lunch"" (NFL) theorem states that ...",2.572277,"The ""no free lunch"" (NFL) theorem for optimiza..."
3,qwen2.5:3b,What is the concept behind Parallel Algorithm ...,Document 1: \n training set as well as for con...,The concept of a Parallel Algorithm Portfolio ...,2.925742,The concept behind a Parallel Algorithm Portfo...
4,qwen2.5:3b,Please provide some approaches to how Parallel...,"Document 1: \n algorithms, and thereby combine...","To construct parallel algorithm portfolios, ef...",3.219646,Several approaches can be used for constructin...
...,...,...,...,...,...,...
235,llama3.2:latest,How can EBMs help detect observations poorly i...,Document 1: \n concatenation of the two curves...,EBMs (Explainable Boosting Machines) can help ...,4.825688,EBMs can help detect observations poorly imput...
236,llama3.2:latest,How can one distinct terms intepretabilitry an...,Document 1: \n our approach shows better resul...,"The terms ""interpretability"" and ""explainabili...",4.310438,Interpretability and explainability are terms ...
237,llama3.2:latest,What issues in machine learning can be categor...,Document 1: \n 2.1 Characterizing Model Bugs.\...,"According to the text, model bugs are categori...",2.143233,Model bugs in machine learning can be categori...
238,llama3.2:latest,What XAI techniques can be heloful in detectin...,Document 1: \n query is generally much smaller...,"Based on the provided context, two XAI (Explai...",4.372954,The context suggests that several explainable ...


In [25]:
df.to_csv("../../data/llm_eval/real_outputs_with_gt.csv", index=False)